<a href="https://colab.research.google.com/github/sarathpanat/CNN-dog-breed-prediction/blob/master/CNN-inception_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
from glob import glob

dog_files = np.array(glob("/content/drive/My Drive/project-dog-classification/project-dog-classification/dogImages/*/*/*"))
dog_train = np.array(glob("/content/drive/My Drive/project-dog-classification/project-dog-classification/dogImages/train/*/*"))

print('There are %d total dog images.' % len(dog_files))
print('There are %d training dog images.' % len(dog_train))

There are 8351 total dog images.
There are 6680 training dog images.


In [ ]:
#models subpackage contains model architectures for image classification
import torch
import torchvision.models as models

import torch.nn as nn

#construct inception model with pre-trained weights
inception = models.inception_v3(pretrained=True, aux_logits=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# move model to GPU if CUDA is available or to cpu
inception = inception.to(device)
print(inception)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [ ]:
import os
from torchvision import datasets
import torchvision.transforms as transforms

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(20),
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/drive/My Drive/project-dog-classification/project-dog-classification/dogImages/'

images_data = {p: datasets.ImageFolder(os.path.join(data_dir, p), 
                                       data_transforms[p]) 
               for p in ['train', 'valid', 'test']}

dataloaders = {p: torch.utils.data.DataLoader(images_data[p], batch_size=64, 
                                              shuffle=True, num_workers=4)
               for p in ['train', 'valid', 'test']}


In [ ]:
from collections import OrderedDict
for param in inception.parameters():
    param.requires_grad = False

fc = nn.Sequential(OrderedDict([
     ('fc',nn.Linear(2048,len(images_data['train'].classes))),
     ('output',nn.LogSoftmax(dim=1))
]))

inception.fc = fc
inception.to(device)

print(inception)


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [ ]:
from torchsummary import summary
summary(inception, (3, 299, 299))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             864
       BatchNorm2d-2         [-1, 32, 149, 149]              64
       BasicConv2d-3         [-1, 32, 149, 149]               0
            Conv2d-4         [-1, 32, 147, 147]           9,216
       BatchNorm2d-5         [-1, 32, 147, 147]              64
       BasicConv2d-6         [-1, 32, 147, 147]               0
            Conv2d-7         [-1, 64, 147, 147]          18,432
       BatchNorm2d-8         [-1, 64, 147, 147]             128
       BasicConv2d-9         [-1, 64, 147, 147]               0
        MaxPool2d-10           [-1, 64, 73, 73]               0
           Conv2d-11           [-1, 80, 73, 73]           5,120
      BatchNorm2d-12           [-1, 80, 73, 73]             160
      BasicConv2d-13           [-1, 80, 73, 73]               0
           Conv2d-14          [-1, 192,

In [ ]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

### TODO: select optimizer
# optimizer_inception = optim.Adam(model_scratch.parameters(), lr=0.001, momentum=0.9)
optimizer_scratch = optim.Adam(inception.parameters(), lr=0.001)

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, device, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU or CPU
            data, target = data.to(device), target.to(device)
            
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            
            loss.backward()
            optimizer.step()
            
            train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU or CPU
            data, target = data.to(device), target.to(device)
            
            ## update the average validation loss
            output = model(data)
            loss = criterion(output, target)
            
            valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Saving model..')
            valid_loss_min = valid_loss
            torch.save(model.state_dict(), save_path)
            
    # return trained model
    return model

In [ ]:
inception2 = train(30, dataloaders, resnet18, optimizer_scratch, 
                      criterion_scratch, device, 'inception2.pt')

Epoch: 1 	Training Loss: 3.329044 	Validation Loss: 0.892375
Saving model..
Epoch: 2 	Training Loss: 1.698378 	Validation Loss: 0.755382
Saving model..
Epoch: 3 	Training Loss: 1.335221 	Validation Loss: 0.687160
Saving model..
Epoch: 4 	Training Loss: 1.216069 	Validation Loss: 0.686021
Saving model..
Epoch: 5 	Training Loss: 1.132868 	Validation Loss: 0.669559
Saving model..
Epoch: 6 	Training Loss: 1.046515 	Validation Loss: 0.629739
Saving model..
Epoch: 7 	Training Loss: 1.031982 	Validation Loss: 0.596398
Saving model..
Epoch: 8 	Training Loss: 0.984553 	Validation Loss: 0.604943
Epoch: 9 	Training Loss: 0.967358 	Validation Loss: 0.645473
Epoch: 10 	Training Loss: 0.967447 	Validation Loss: 0.672472
Epoch: 11 	Training Loss: 0.910595 	Validation Loss: 0.651557
Epoch: 12 	Training Loss: 0.932173 	Validation Loss: 0.633538
Epoch: 13 	Training Loss: 0.909841 	Validation Loss: 0.603627
Epoch: 14 	Training Loss: 0.904366 	Validation Loss: 0.756788
Epoch: 15 	Training Loss: 0.916865 	

In [ ]:
inception2.load_state_dict(torch.load('inception2.pt'))

<All keys matched successfully>

In [ ]:
def test(loaders, model, criterion, device):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU or CPU
        data, target = data.to(device), target.to(device)
        
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [ ]:
test(dataloaders, inception, criterion_scratch, device)

Test Loss: 0.734207


Test Accuracy: 81% (684/836)


In [ ]:
inception_3 = train(100, dataloaders, inception, optimizer_scratch, 
                      criterion_scratch, device, 'inception_3.pt')

Epoch: 1 	Training Loss: 3.334334 	Validation Loss: 1.042374
Saving model..
Epoch: 2 	Training Loss: 1.708880 	Validation Loss: 0.681924
Saving model..
Epoch: 3 	Training Loss: 1.337258 	Validation Loss: 0.793185
Epoch: 4 	Training Loss: 1.221940 	Validation Loss: 0.656210
Saving model..
Epoch: 5 	Training Loss: 1.107261 	Validation Loss: 0.613280
Saving model..
Epoch: 6 	Training Loss: 1.068344 	Validation Loss: 0.611767
Saving model..
Epoch: 7 	Training Loss: 1.025156 	Validation Loss: 0.618616
Epoch: 8 	Training Loss: 1.010044 	Validation Loss: 0.621361
Epoch: 9 	Training Loss: 0.963864 	Validation Loss: 0.627966
Epoch: 10 	Training Loss: 0.962499 	Validation Loss: 0.593399
Saving model..
Epoch: 11 	Training Loss: 0.934811 	Validation Loss: 0.647555
Epoch: 12 	Training Loss: 0.906864 	Validation Loss: 0.607858
Epoch: 13 	Training Loss: 0.902198 	Validation Loss: 0.624786
Epoch: 14 	Training Loss: 0.897774 	Validation Loss: 0.671481
Epoch: 15 	Training Loss: 0.881846 	Validation Loss

In [ ]:
test(dataloaders, inception_3, criterion_inception, device)

Test Loss: 0.779765


Test Accuracy: 83% (698/836)


In [ ]:
import torch.optim as optim

### TODO: select loss function
criterion_exp = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_exp = optim.Adam(inception.parameters())

In [ ]:
import math
def train_exp(n_epochs, loaders, model, optimizer, criterion, device, save_path, initial_rate = 0.1, k=0.1):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        #Exponential Decay
        #lr = initial_rate * exp(-k*t)
        #initial_rate = 0.1
        #k = 0.1

        lr = initial_rate * math.exp(-k*epoch)    
        print('The learning rate was set to {}.'.format(lr))
        for param_group in optimizer.param_groups:
                    param_group['lr'] = lr
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            
            # move to GPU or CPU
            data, target = data.to(device), target.to(device)
            
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            
            loss.backward()
            optimizer.step()
            
            train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU or CPU
            data, target = data.to(device), target.to(device)
            
            ## update the average validation loss
            output = model(data)
            loss = criterion(output, target)
            
            valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Saving model..')
            valid_loss_min = valid_loss
            torch.save(model.state_dict(), save_path)
            
    # return trained model
    return model

In [ ]:
inceptionv3 = train_exp(100, dataloaders, inception, optimizer_exp, 
                      criterion_exp, device, 'inceptionv3.pt')

The learning rate was set to 0.09048374180359596.
Epoch: 1 	Training Loss: 70.206329 	Validation Loss: 70.682030
Saving model..
The learning rate was set to 0.0818730753077982.
Epoch: 2 	Training Loss: 72.522659 	Validation Loss: 67.528595
Saving model..
The learning rate was set to 0.0740818220681718.
Epoch: 3 	Training Loss: 69.801796 	Validation Loss: 66.645844
Saving model..
The learning rate was set to 0.06703200460356394.
Epoch: 4 	Training Loss: 68.832458 	Validation Loss: 70.043900
The learning rate was set to 0.06065306597126335.
Epoch: 5 	Training Loss: 67.496941 	Validation Loss: 82.717194
The learning rate was set to 0.05488116360940264.
Epoch: 6 	Training Loss: 65.904480 	Validation Loss: 81.053513
The learning rate was set to 0.04965853037914095.
Epoch: 7 	Training Loss: 63.521393 	Validation Loss: 67.319901
The learning rate was set to 0.044932896411722156.
Epoch: 8 	Training Loss: 63.913483 	Validation Loss: 64.592857
Saving model..
The learning rate was set to 0.040656

In [ ]:
inception.load_state_dict(torch.load('inceptionv3.pt'))

<All keys matched successfully>

In [ ]:
test(dataloaders, inceptionv3, criterion_exp, device)

Test Loss: 50.441177


Test Accuracy: 84% (706/836)
